# Logistic Regression Using Food Items Dataset

The dataset contains 17 features and 1 target:          
 1   Total Fat            
 2   Saturated Fat        
 3   Monounsaturated Fat  
 4   Polyunsaturated Fat  
 5   Trans Fat            
 6   Cholesterol           
 7   Sodium               
 8   Total Carbohydrate   
 9   Dietary Fiber        
 10  Sugars               
 11  Sugar Alcohol         
 12  Protein             
 13  Vitamin A             
 14  Vitamin C           
 15  Calcium               
 16  Iron   
 17  Calories  

 
Target: class with three categories:'In Moderation','Less Often','More Often'

Problem: Predict whether a food item is Healthy, In Moderation, or Unhealthy based on its nutritional values.

#### **Importing necessary libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,ConfusionMatrixDisplay, precision_recall_fscore_support, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.multiclass import OneVsRestClassifier

import warnings
warnings.filterwarnings("ignore")


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:

        print(os.path.join(dirname, filename))

In [ ]:
#data = pd.read_csv(r"/kaggle/input/food-items/food_items.csv")
data = pd.read_csv(r"/kaggle/input/food-items-dataset/food_items.csv")
data.head()

# Data Exploration

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum() #no null values

In [ ]:
data.duplicated().sum() #duplicated records

In [ ]:
food_df = data.copy()

In [ ]:
feature_cols = list(food_df.iloc[:, :-1].columns)
feature_cols

In [ ]:
food_df.describe() # Fetaures need to be scaled

In [ ]:
food_df[food_df.duplicated()]

In [ ]:
food_df.nunique() # want to make sure of the number of each

In [ ]:
food_df[food_df['Calories'] == 110]

In [ ]:
# Correlation Matrix
numeric_df = food_df.drop(columns='class')

corr_matrix = numeric_df.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=0.5)
plt.title("Correlation Matrix of Food Features")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
print("strong positive correlations:")
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > 0.8:
            print(f"{corr_matrix.columns[i]} and {corr_matrix.columns[j]}: {corr_matrix.iloc[i, j]:.2f}")

In [ ]:
#outliers possible issue?
outlier = {}

for feature in numeric_df:
    Q1 = food_df[feature].quantile(0.25)
    Q3 = food_df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find outliers
    outliers = food_df[(food_df[feature] < lower_bound) | (food_df[feature] > upper_bound)]

    # Store stats
    outlier[feature] = {
        'num_outliers': outliers.shape[0],
        'percent_outliers %':  outliers.shape[0] / data.shape[0] *100 ,
        'min_outlier': outliers[feature].min() ,
        'max_outlier': outliers[feature].max() 
    }

outlier = pd.DataFrame(outlier).T
outlier

# Data Cleaning

### Removing Duplicates

In [ ]:
# No.of duplicates
food_duplicates = food_df.duplicated().sum()
print(f"Number of duplicate rows: {food_duplicates}")

In [ ]:
# Find duplicated rows
duplicates = food_df[food_df.duplicated()]

# Count how many of each class are in the duplicates
class_counts_in_duplicates = duplicates['class'].value_counts()

print("Class distribution in duplicate rows:")
print(class_counts_in_duplicates)

In [ ]:
# Removing duplicates
food_df = food_df.drop_duplicates()
food_df.shape

In [ ]:
# Resetting index
food_df = food_df.reset_index(drop=True)

In [ ]:
food_df.tail(3)

In [ ]:
# Display the relative proportion of each class 
food_df['class'].value_counts(normalize=True)

In [ ]:
food_df['class'].value_counts(normalize=True).plot.bar(color=['green', 'blue', 'red'])
plt.show()

#### As we can see from the bar chart above, this dataset has three classes: 
- `In Moderation`, `Less Often`, and `More Often`.
-  The three labels are **imbalanced**.
-  For diabetic patients, most food items are in the `In Moderation` and `Less Often` categories. This makes diabetes diet management very hard, so we could build a machine learning model to help patients choose their food.

#### We have three labels meaning our logistic regression model will be multinomial with three classes.

#### `Multinomial logistic regression` model is different from the `one-vs-rest` binary logistic regression. 
- For `one-vs-rest` schema, you need to train an **independent classifier for each class.**
- For example, you need a `More Often` classifier to differentiate a food item between `More Often` and **Not More Often** (or, `In Moderation` and `Less Often`).

# Data Preprocessing

In [ ]:
X = food_df.drop(columns='class')
y = food_df['class']

pd.set_option("mode.use_inf_as_na", True)
# distribution of each numerical feature
plt.figure(figsize=(20, len(X.columns) * 4))
for i, col in enumerate(X.columns):
    plt.subplot(len(X.columns), 3, i + 1)
    sns.histplot(X[col], kde=True, bins=30)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

#Skewness is apparent 

### Applying Log Transformation

In [ ]:
# Identify numeric columns with absolute skewness greater than the specified threshold 50%
def normalize_skewed(df, skew_threshold=0.5):
    skewed_cols = df.select_dtypes(include=[np.number]).apply(lambda x: x.skew()).abs()
    skewed_cols = skewed_cols[skewed_cols > skew_threshold].index.tolist()

    # Apply log(1 + x) transformation to reduce skewness in those columns
    df[skewed_cols] = df[skewed_cols].apply(lambda x: np.log1p(x))
    return df

In [ ]:
X_log = X.copy()
X_log = normalize_skewed(X_log)

In [ ]:
# Comparing before and after log transformation
def compare_distribution(original, transformed, features, n_cols=3):
    n_rows = (len(features) + n_cols - 1) // n_cols
    plt.figure(figsize=(n_cols * 5, n_rows * 4))

    for i, col in enumerate(features):
        plt.subplot(n_rows, n_cols, i + 1)
        sns.kdeplot(original[col], label='Original', color='blue')
        sns.kdeplot(transformed[col], label='Transformed', color='green')
        plt.title(f"{col}")
        plt.legend()

    plt.tight_layout()
    plt.show()

compare_distribution(X, X_log, X.columns)

### Target Encoding

In [ ]:
# Fit the encoder to the target labels and transform them into integer values 0,1,2
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# observe mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Class Mapping:", label_mapping)

### Data Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size = 0.2, random_state=41, stratify=y_encoded)

### Preprocessing Pipeline

In [ ]:
log_transformer = FunctionTransformer(np.log1p, validate=True)

pipeline = Pipeline([
    ('log', log_transformer),
    ('scaler', StandardScaler())
])

X_train_scaled = pipeline.fit_transform(X_train)
# transforming X_test
X_test_scaled = pipeline.transform(X_test)

In [ ]:
print(f"Training dataset shape, X_train: {X_train_scaled.shape}, y_train: {y_train.shape}")

In [ ]:
print(f"Testing dataset shape, X_test: {X_test_scaled.shape}, y_test: {y_test.shape}")

### Handling imbalance

In [ ]:
food_df['class'].value_counts(normalize=True)

In [ ]:
print("Original class distribution:")
class_counts = pd.Series(y_train).value_counts()
plt.figure(figsize=(4, 4))
class_counts.plot(kind='bar')
plt.title('Original Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()
class_counts

In [ ]:
# Method 1: SMOTE (Synthetic Minority Oversampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)
print("SMOTE distribution:")
class_counts = pd.Series(y_train_smote).value_counts()
plt.figure(figsize=(4, 4))
class_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()
class_counts

In [ ]:
# Method 2: Random Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_scaled, y_train)
print("Random Undersampling distribution:")
class_counts = pd.Series(y_train_rus).value_counts()
plt.figure(figsize=(4, 4))
class_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()
class_counts

# Multinomial Logistic Regression

#### 1. Without Regularization

In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp)
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [ ]:
def mlr(X_train, y_train, X_test):
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    return y_pred , clf

**Since this is a health related issues ----> recall is more important (catch all foods that are actually unhealthy)**

In [ ]:
y_pred, clf = mlr(X_train_scaled, y_train, X_test_scaled)
cf = confusion_matrix(y_test, y_pred)
print("original")
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=clf.classes_)
disp.plot()
plt.show()
evaluate_metrics(y_test, y_pred)

In [ ]:
y_pred2 ,clf = mlr(X_train_rus, y_train_rus, X_test_scaled)
cf = confusion_matrix(y_test, y_pred2)
print("random undersampling")
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=clf.classes_)
disp.plot()
plt.show()
evaluate_metrics(y_test, y_pred2)

In [ ]:
y_pred3 ,clf = mlr(X_train_smote, y_train_smote, X_test_scaled)
cf = confusion_matrix(y_test, y_pred3)
print("SMOTE")
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=clf.classes_)
disp.plot()
plt.show()
evaluate_metrics(y_test, y_pred3)


**SMOTE THE BEST ?{'accuracy': 0.7902930402930403, 'recall': array([0.76575729, 0.790111  , 0.99230769]), 'precision': array([0.80276134, 0.80143296, 0.66839378]), 'f1score': array([0.78382282, 0.79573171, 0.79876161])}**


**Prior to balancing, the confusion matrix showed heavy bias toward the majority class. Post-balancing, classification became more uniform.**

**SMOTE improved the model's recall and F1-score for underrepresented classes (overfitting?)**

### regularized model L1

In [ ]:
# Multinomial LR with L1 regularization - SMOTE
print("Multinomial LR Lasoo - SMOTE")
mlr_l1_smote = LogisticRegression(penalty='l1', C=0.1, multi_class='multinomial',solver='saga', max_iter=2000, random_state=42)
mlr_l1_smote.fit(X_train_smote, y_train_smote)

y_pred_mlr_l1_smote = mlr_l1_smote.predict(X_test_scaled)
y_proba_mlr_l1_smote = mlr_l1_smote.predict_proba(X_test_scaled)

print(evaluate_metrics(y_test, y_pred_mlr_l1_smote))
print(classification_report(y_test, y_pred_mlr_l1_smote, target_names=label_encoder.classes_))

# One-vs-Rest Logistic Regression 

In [ ]:
print("One-vs-Rest LR - Original Data")
ovr_original = OneVsRestClassifier(LogisticRegression(solver='liblinear', max_iter=1000, random_state=42))
ovr_original.fit(X_train_scaled, y_train)
y_pred_ovr_orig = ovr_original.predict(X_test_scaled)

acc_ovr_orig = evaluate_metrics(y_test, y_pred_ovr_orig)
print(acc_ovr_orig)


In [ ]:
print("One-vs-Rest LR - SMOTE Balanced")
ovr_smote = OneVsRestClassifier(LogisticRegression(solver='liblinear', max_iter=1000, random_state=42))
ovr_smote.fit(X_train_smote, y_train_smote)

y_pred_ovr_smote = ovr_smote.predict(X_test_scaled)
y_proba_ovr_smote = ovr_smote.predict_proba(X_test_scaled)

# Use the correct evaluate_metrics function
acc_ovr_smote = evaluate_metrics(y_test, y_pred_ovr_smote)
print(acc_ovr_smote)


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

# Binarize the output labels for ROC AUC (required for multi-class ROC)
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
n_classes = y_test_bin.shape[1]

def plot_roc_auc_curve(clf, X_test, y_test_bin, model_name):
    if not hasattr(clf, "predict_proba"):
        print(f"{model_name} does not support predict_proba.")
        return
    
    y_score = clf.predict_proba(X_test)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot all ROC curves
    plt.figure()
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
# Multinomial LR - Original
y_pred, clf = mlr(X_train_scaled, y_train, X_test_scaled)
plot_roc_auc_curve(clf, X_test_scaled, y_test_bin, "Multinomial LR - Original")

# Multinomial LR - RUS
y_pred2, clf2 = mlr(X_train_rus, y_train_rus, X_test_scaled)
plot_roc_auc_curve(clf2, X_test_scaled, y_test_bin, "Multinomial LR - RUS")

# Multinomial LR - SMOTE
y_pred3, clf3 = mlr(X_train_smote, y_train_smote, X_test_scaled)
plot_roc_auc_curve(clf3, X_test_scaled, y_test_bin, "Multinomial LR - SMOTE")

# Multinomial LR L1 - SMOTE
plot_roc_auc_curve(mlr_l1_smote, X_test_scaled, y_test_bin, "Multinomial LR L1 - SMOTE")

# One-vs-Rest - Original
plot_roc_auc_curve(ovr_original, X_test_scaled, y_test_bin, "OvR LR - Original")

# One-vs-Rest - SMOTE
plot_roc_auc_curve(ovr_smote, X_test_scaled, y_test_bin, "OvR LR - SMOTE")


### Features Importance

In [ ]:
clf.coef_

#### The `coef_` is a coefficients list with three elements, one element is the actual coefficent for class 0, 1, 2.
- To better analyze the coefficients, use three utility methods to sort and visualize them.

In [ ]:
# Extract and sort feature coefficients
def get_feature_coefs(regression_model, label_index, columns):
    coef_dict = {}
    for coef, feat in zip(regression_model.coef_[label_index, :], columns):
        if abs(coef) >= 0.01:
            coef_dict[feat] = coef
    # Sort coefficients
    coef_dict = {k: v for k, v in sorted(coef_dict.items(), key=lambda item: item[1])}
    return coef_dict

# bar colors based on if value is negative or positive
def get_bar_colors(values):
    color_vals = []
    for val in values:
        if val <= 0:
            color_vals.append('r')
        else:
            color_vals.append('g')
    return color_vals

# visualizing coefficients
def visualize_coefs(coef_dict):
    features = list(coef_dict.keys())
    values = list(coef_dict.values())
    y_pos = np.arange(len(features))
    color_vals = get_bar_colors(values)
    plt.rcdefaults()
    fig, ax = plt.subplots()
    ax.barh(y_pos, values, align='center', color=color_vals)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(features)
    # labels read top-to-bottom
    ax.invert_yaxis()  
    ax.set_xlabel('Feature Coefficients')
    ax.set_title('')
    plt.show()

#### Coefficients for Classes

In [ ]:
# coefficents for Class 1, `Less Often`
coef_dict = get_feature_coefs(clf, 1, feature_cols)
visualize_coefs(coef_dict)

#### Unhealthy nutrients such as `Saturated Fat`, `Sugars`, `Cholesterol`, `Total Fat`, other fats., have high positive coefficients.
- Food items containing unhealthy nutrients will have **higher coeficients** and will be more likely to be categorized in the 'Less Often' class.

In [ ]:
# coefficents for Class 2, `More Often` ==> healthiest options
coef_dict = get_feature_coefs(clf, 2, feature_cols)
visualize_coefs(coef_dict)

#### 2.  `L1-Regularized` Multinomial Logistic Regression

In [ ]:
l2_model = LogisticRegression(
    penalty='l2',
    multi_class='multinomial',
    solver='lbfgs',  # works with l2
    C=0.01,            # inverse of regularization strength (lambda)
    random_state=42, 
    max_iter=1000
)
l2_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_l2 = l2_model.predict(X_test_scaled)

In [ ]:
evaluate_metrics(y_test, y_pred_l2)

In [ ]:
print(classification_report(y_test, y_pred_l2, target_names=label_encoder.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred_l2)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Multinomial Logistic Regression")
plt.show()

In [ ]:
# coefficents for Class 1, `Less Often`
coef_dict = get_feature_coefs(clf, 2, feature_cols)
visualize_coefs(coef_dict)

#### 3.  `L1-Regularized` Multinomial Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

l1_model = LogisticRegression(
    penalty='l1',
    solver='saga',               
    multi_class='multinomial',   
    C=0.01,                      
    max_iter=5000,              
    random_state=42
)

l1_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred_l1 = l1_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_l1))
print(classification_report(y_test, y_pred_l1, target_names=label_encoder.classes_))

In [ ]:
# Which features are used
# non-zero coefficients per class
nonzero_counts = np.sum(l1_model.coef_ != 0, axis=1)
print("Number of non-zero coefficients per class:", nonzero_counts)

In [ ]:
feature_names = X_train.columns  # list of original feature names
coefs = l1_model.coef_           # shape: (n_classes, n_features)

In [ ]:
#class labels (e.g., 'In Moderation', 'Less Often', 'More Often')
class_labels = label_encoder.classes_

# loop over each class
for i, class_label in enumerate(class_labels):
    coef_i = coefs[i]
    nonzero_mask = coef_i != 0
    selected_features = feature_names[nonzero_mask]
    selected_coefs = coef_i[nonzero_mask]

    plt.figure(figsize=(10, 5))
    sns.barplot(x=selected_coefs, y=selected_features, palette='coolwarm')
    plt.title(f"Non-Zero Coefficients for Class: {class_label}")
    plt.xlabel("Coefficient Value")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()